In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
stout_url = 'https://www.danmurphys.com.au/dm/navigation/navigation_results.jsp?params=fh_location%3D%2F%2Fcatalog01%2Fen_AU%2Fcategories%3C%7Bcatalog01_2534374302025910%7D%2Fwebbeertype%3D%7Bdark20beer%7D%26fh_view_size%3D120%26fh_sort%3D-sales_value_30_days%26fh_modification%3D%2528secondid%253C%257Bdm_mystery57%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_mystery52%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_757646%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_610528%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_786481%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_616722%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_561685%257D%252Cblocked%2529%252C%2528instock%253D0%252Cbottom%2529&resetnav=false&storeExclusivePage=false'

In [4]:
headers = requests.utils.default_headers()

headers.update(
    {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    }
)

In [5]:
result = requests.get(stout_url, headers=headers)
stout_page = result.content
with open('stouts.html','w') as f:
    to_write = stout_page.decode("utf-8", "ignore")
    f.write(to_write)

In [6]:
f = open('stouts.html')
soup = BeautifulSoup(f, 'html.parser')

In [7]:
products = soup.find_all("h3", { "class" : "title-product" }) 
len(products)

44

In [8]:
untapped_search_url = 'https://untappd.com/search'

In [9]:
def find_matching_beers(product_name):
    """
    Searches untappd with the given product name and returns results
    
    Args:
        product_name (string): The product name to search untappd for
        
    Returns:
        Array of matching html elements for untappd search results
    """
    result = requests.get(untapped_search_url, params={'q': product_name.replace('&','') })
    #print(result.url)
    soup = BeautifulSoup(result.content, 'html.parser')
    matching_beers = soup.find_all('div', {'class': 'beer-item'})
    return matching_beers

def strip_stop_words(product_name, stop_words):
    """
    Strips out given stop words from product name
    
    Args:
        product_name (string): The product name to strip stop words from
        stop_words (list): The stop words to strip from the product name
    
    Returns:
        (string) the product name with stop words stripped from it
    """
    product_name_stripped = product_name
    for word in stop_words:
        product_name_stripped = product_name_stripped.replace(word, '')
    return product_name_stripped

In [18]:
data = []
for product in products:
    brand = product.span.text.strip()
    product_name_full = product.a['title']
    product_name = product_name_full[:-6].replace('Cans','').replace('Bottles','').strip()
    product_name_alone = product_name.replace(brand,'').strip()
    print(brand, product_name_alone)
    
    stop_words = ['&']
    matching_beers = find_matching_beers(strip_stop_words(product_name, stop_words))
    if(len(matching_beers) == 0):
        stop_words+=['Domestic Stout','&','and','The']
        matching_beers = find_matching_beers(strip_stop_words(product_name, stop_words))
        if(len(matching_beers) == 0):
            stop_words+=['The Black Box Of Dark Ales','Stout', 'Imperial','Ale','Polish', 
                         'Beer','Dark Lager','Russian','-', 'Doppelbock', 'Dark']
            polish = product_name.replace('Kozel Dark','Kozel Černý')
            matching_beers = find_matching_beers(strip_stop_words(polish, stop_words))
    
    count = 0
    for beer in matching_beers:
        count+=1
        name = beer.find('p', {'class', 'name'}).a.text.strip()
        rating = beer.find('span', {'class', 'num'}).text.replace('(','').replace(')','').strip()
        brewery = beer.find('p', {'class', 'brewery'}).a.text.strip()
        if (float(rating) > 0.1 
            and (
                brewery.split()[0] == brand.split()[0] 
                or name.split()[0] == brand.split()[0]
                or brand in ['Zywiec', 'Youngs']
                or brewery in ['BrewDog']
            )):
            print('*', brewery, name, rating)
            if count == 1:
                data.append([brewery,name,rating])
    print()

Tooheys Old
* Tooheys Brewing Old 3.138

White Rabbit Dark Ale
* White Rabbit Dark Ale 3.611
* White Rabbit Dark Ale Brewers Cut 3.845
* White Rabbit Kinda Dark Ale (Time Out Collab) 3.297
* White Rabbit Dark Christmas Pudding Ale 3.232

Monteith's Black Beer
* Monteith's Brewing Co. Monteith's Black Beer 3.483

Coopers Dark Ale
* Coopers Brewery Dark Ale 3.364
* Coopers Brewery Limited - Homebrews DIY Beer: Dark Ale 3.297

James Squire Jack of Spades Porter
* Malt Shovel Brewery James Squire Jack of Spades Porter 3.467

Kozel Dark Beer
* Pivovar Velké Popovice Kozel Černý / Dark 3.335
* Pivovar Velké Popovice Kozel Černý nefiltrovaný / Black unfiltered 3.567

Stone Xocoveza Choc Mocha Stout
* Stone Brewing Stone Xocoveza 4.156
* Stone Brewing Stone Xocoveza (2016) 4.157

Zywiec Polish Porter
* Grupa Żywiec Żywiec Porter 3.573

Youngs Double Chocolate Stout
* Charles Wells Brewery Young's Double Chocolate Stout 3.719

Bridge Road Brewers Robust Porter
* Bridge Road Brewers Robust Porte

In [19]:
import pandas as pd

In [20]:
df = pd.DataFrame(data=data, columns=['Brewery', 'Name', 'Rating'])
df

,Brewery,Name,Rating
0,Tooheys Brewing,Old,3.138
1,White Rabbit,Dark Ale,3.611
2,Monteith's Brewing Co.,Monteith's Black Beer,3.483
3,Coopers Brewery,Dark Ale,3.364
4,Malt Shovel Brewery,James Squire Jack of Spades Porter,3.467
5,Pivovar Velké Popovice,Kozel Černý / Dark,3.335
6,Stone Brewing,Stone Xocoveza,4.156
7,Grupa Żywiec,Żywiec Porter,3.573
8,Charles Wells Brewery,Young's Double Chocolate Stout,3.719
9,Bridge Road Brewers,Robust Porter,3.765


In [21]:
df.sort_values(by='Rating', ascending=False, inplace=True)
df

,Brewery,Name,Rating
23,Nail Brewing Australia,Clout Stout,4.309
16,Mountain Goat Beer,Barrel Breed Imperial Stout 2017,4.195
6,Stone Brewing,Stone Xocoveza,4.156
38,Sierra Nevada Brewing Co.,Narwhal Imperial Stout,3.986
31,4 Pines Brewing Company,Keller Door: Double Cascadian Dark Ale,3.964
33,Mountain Goat Beer,Cross Breed: Seedy Goat Coffee Porter (2015),3.952
35,4 Pines Brewing Company,Keller Door: Imperial India Brown Ale,3.945
30,Mountain Goat Beer,Rare Breed: Captain Amylase Rum Porter,3.901
36,BrewDog,Tactical Nuclear Penguin,3.9
37,4 Pines Brewing Company,Keller Door: Oaked Baltic Porter,3.857


In [25]:
df.reset_index(drop=True, inplace=True)
df.index = df.index + 1
df

,Brewery,Name,Rating
1,Nail Brewing Australia,Clout Stout,4.309
2,Mountain Goat Beer,Barrel Breed Imperial Stout 2017,4.195
3,Stone Brewing,Stone Xocoveza,4.156
4,Sierra Nevada Brewing Co.,Narwhal Imperial Stout,3.986
5,4 Pines Brewing Company,Keller Door: Double Cascadian Dark Ale,3.964
6,Mountain Goat Beer,Cross Breed: Seedy Goat Coffee Porter (2015),3.952
7,4 Pines Brewing Company,Keller Door: Imperial India Brown Ale,3.945
8,Mountain Goat Beer,Rare Breed: Captain Amylase Rum Porter,3.901
9,BrewDog,Tactical Nuclear Penguin,3.9
10,4 Pines Brewing Company,Keller Door: Oaked Baltic Porter,3.857
